# 1. Introduction

## Predicting Player Elo from Opening Moves

### Project Overview
The aim is, to use the Lichess Standard Rated Games Dataset, publicly available on Huggingface (https://huggingface.co/datasets/Lichess/standard-chess-games) to build regression models that predict a player’s Elo rating based solely on the first _n_ moves of a game. By focusing on opening patterns, our goal is to uncover which early-game features best correlate with player strength. The underlying hypothesis is that players more experienced with chess will adhere more strictly to certain established chess openings.

### Data Source
- **Dataset**: Lichess Standard Rated Games
    - Structured by Year and Month
    - Around 20-30GB of Data per month
    - Provided as .parquet files, easily convertable to pgn
- **Key Fields**:
  - `UTCDate`, `UTCTime`
  - `White`, `Black`
  - `WhiteElo`, `BlackElo` (target variables)
  - `movetext` (contains PGN moves)

### Pipeline
1. **Data Extraction**
   - Stream only the desired month’s Parquet partitions
      - Find out what amount of data is managable
   - Filter out incomplete or abnormal games
   - Reduce DataFrame to key fields for analysis
2. **Feature Engineering**
   - Parse `movetext` to extract the first _n_ half‑moves (e.g. 10 plies)
   - Encode moves as categorical sequences, opening ECO codes, or vectorized embeddings
   - Optional: Include auxiliary features: time control, rating differences, termination type
3. **Modeling**
   - Train regression algorithms (e.g. linear regression, random forest)
   - Cross‑validate on different `_n_` to see how many moves are needed for accurate Elo estimates
4. **Evaluation**
   - Measure performance via RMSE and R² on held‑out data
   - Analyze feature importances to identify key opening moves or patterns

# 2. Data Loading and Exploration

In [1]:
from datasets import load_dataset
import pandas as pd


In [2]:
# pick your year & month
year, month = 2015, 2
month_str = f"{month:02d}"

# define date period to be downloaded
data_files = {
    "games": f"https://huggingface.co/datasets/lichess/standard-chess-games/resolve/main/data/year={year}/month={month_str}/*.parquet"
}

ds = load_dataset(
    "parquet",
    data_files=data_files,
    split="games",
    streaming=True
)

# load first 10k entries into DataFrame
df = pd.DataFrame([x for _, x in zip(range(10_000), ds)])
print(df.head())

                                               Event  \
0  Rated Bullet tournament https://lichess.org/to...   
1                                   Rated Blitz game   
2                               Rated Classical game   
3                                   Rated Blitz game   
4                                  Rated Bullet game   

                           Site           White           Black Result  \
0  https://lichess.org/q1fO8MLl      andryutz10  AwareTenacious    0-1   
1  https://lichess.org/QE4zHivV  cvetlicnivitez      gravihouse    0-1   
2  https://lichess.org/Ooo36zZs       grillo131        enzo9607    1-0   
3  https://lichess.org/QK5egQTl      lugalbanda   Amilcar_Hdez1    1-0   
4  https://lichess.org/hGGZL4Um        TrainMan   Doctor_WHO_13    1-0   

  WhiteTitle BlackTitle  WhiteElo  BlackElo  WhiteRatingDiff  BlackRatingDiff  \
0       None       None      1500      1642              NaN              NaN   
1       None       None      1498      1445            -

In [ ]:
def row_to_pgn(r):
    tags = [
        f'[Event "{r.Event}"]', f'[Site "{r.Site}"]',
        f'[UTCDate "{r.UTCDate}"]', f'[UTCTime "{r.UTCTime}"]',
        f'[White "{r.White}"]', f'[Black "{r.Black}"]',
        f'[Result "{r.Result}"]'
    ]
    return "\n".join(tags) + "\n\n" + r.movetext + "\n\n"

pgns = df.apply(row_to_pgn, axis=1).tolist()
